In [6]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

def obtener_titulares_financieros(empresa, limite=10):
    # Construir URL de búsqueda en Google News (versión actual)
    query = urllib.parse.quote(f"{empresa} finanzas OR acciones OR resultados OR bolsa")
    url = f"https://news.google.com/search?q={query}&hl=es-419&gl=CO&ceid=CO:es"

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        ),
        "Accept-Language": "es-ES,es;q=0.9,en;q=0.8",
        "Referer": "https://www.google.com/"
    }

    # Solicitud HTTP
    r = requests.get(url, headers=headers, timeout=10)
    if r.status_code != 200:
        print(f"⚠️ Error {r.status_code} al acceder a Google News.")
        return []

    soup = BeautifulSoup(r.text, "html.parser")

    # Google News ahora usa etiquetas <a> con "DY5T1d" o "JtKRv"
    enlaces = soup.select("a.DY5T1d, a.JtKRv")

    resultados = []
    for enlace in enlaces[:limite]:
        titulo = enlace.get_text(strip=True)
        href = enlace.get("href")
        if href.startswith("./"):
            href = "https://news.google.com" + href[1:]  # convertir a URL completa
        resultados.append({"titulo": titulo, "link": href})

    return resultados


# Ejemplo de uso
if __name__ == "__main__":
    empresa = input("Ingrese el nombre de la empresa: ")
    titulares = obtener_titulares_financieros(empresa)

    if titulares:
        print(f"\n📰 Titulares financieros sobre '{empresa}':\n")
        for i, t in enumerate(titulares, 1):
            print(f"{i}. {t['titulo']}")
            print(f"   🔗 {t['link']}\n")
    else:
        print("No se encontraron titulares.")



📰 Titulares financieros sobre 'tesla':

1. Tesla Motors
   🔗 https://news.google.com/read/CBMiYkFVX3lxTFBtbHBiSWZHc1l4Tl9yTDhwYTZDUXo4TXExOGZUdXJ5dldJeWJrVTlRY2NHeTVmMVdPU004NzFENGltZU02Mzd4MDJ1TlRNVXM5Mjg1UWQ1VTBXeURzNXRDandB?hl=es-419&gl=CO&ceid=CO%3Aes-419

2. Accionistas de Tesla aprueban el paquete salarial de US$ 1 billón de Musk, lo que podría convertirlo en el primer billonario
   🔗 https://news.google.com/read/CBMilAFBVV95cUxPM2RhLXhSUlJGYlJCSHFVRm5wTkpjWnBKZEdiaFdoV21ScWd4bmIyY1hyNEZ3dDg5d0RLVGxHRktmXzBLNy0yVG5SS19HQTA3VlZJX3BsYzM4dWtvX19jb25ZR0ZsckU4enZsaUozdjRlYy04ZlpuVUtmTktrcjQwRTVWdmlMZW55Ujk2eWxHVFBRNFRx?hl=es-419&gl=CO&ceid=CO%3Aes-419

3. Los accionistas de Tesla aprueban pagarle a Elon Musk un paquete salarial de US$1 billón si cumple sus objetivos
   🔗 https://news.google.com/read/CBMiW0FVX3lxTFBTRmVTb2lUcGEzNF9KcnBEd2J4V2puM2Itay1ZZlFUSXNUeXpXZ0JzZl9pREZxZWY2Sjl2Ync5amdjdTdJdEFQdDZoMEZIR2VGZGJtWFN1eUNodlHSAWBBVV95cUxQVHJCSFkzeDJ1TVBtU283R2tab2pHZVhxNW1lMnhiY21xMzB

In [2]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import pandas as pd

def obtener_titulares_financieros_df(empresa, limite=20):
    # Query en inglés (para modelos NLP entrenados en inglés)
    query = urllib.parse.quote(f"{empresa} finance OR stock OR earnings OR market")
    url = f"https://news.google.com/search?q={query}&hl=en&gl=US&ceid=US:en"

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        ),
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/"
    }

    r = requests.get(url, headers=headers, timeout=10)
    if r.status_code != 200:
        print(f"⚠️ Error {r.status_code} accessing Google News.")
        return pd.DataFrame()

    soup = BeautifulSoup(r.text, "html.parser")

    # Titulares (Google News actual)
    enlaces = soup.select("a.DY5T1d, a.JtKRv")

    data = []
    for enlace in enlaces[:limite]:
        titulo = enlace.get_text(strip=True)
        href = enlace.get("href")
        if href.startswith("./"):
            href = "https://news.google.com" + href[1:]
        data.append({
            "company": empresa,
            "title": titulo,
            "link": href
        })

    # Convertir a DataFrame
    df = pd.DataFrame(data)
    return df


# Ejemplo de uso
if __name__ == "__main__":
    empresa = input("Enter company name: ")
    df = obtener_titulares_financieros_df(empresa, limite=15)

    if not df.empty:
        print("\n📰 DataFrame of financial headlines:\n")
        print(df.head())
    else:
        print("No headlines found.")



📰 DataFrame of financial headlines:

  company                                              title  \
0   tesla  Tesla’s Cybertruck Head Exits. What It Means f...   
1   tesla  Tesla Loses Key Execs, China Sales Tumble, But...   
2   tesla  Veteran analyst delivers bold twist on Tesla s...   
3   tesla  Tesla chair says the company could be on the c...   
4   tesla  Elon Musk's pay package is a clear signal that...   

                                                link  
0  https://news.google.com/read/CBMi_wJBVV95cUxQZ...  
1  https://news.google.com/read/CBMiogFBVV95cUxPQ...  
2  https://news.google.com/read/CBMimAFBVV95cUxPb...  
3  https://news.google.com/read/CBMixAFBVV95cUxNV...  
4  https://news.google.com/read/CBMiygFBVV95cUxNQ...  


In [3]:
df

,company,title,link
0,tesla,Tesla’s Cybertruck Head Exits. What It Means f...,https://news.google.com/read/CBMi_wJBVV95cUxQZ...
1,tesla,"Tesla Loses Key Execs, China Sales Tumble, But...",https://news.google.com/read/CBMiogFBVV95cUxPQ...
2,tesla,Veteran analyst delivers bold twist on Tesla s...,https://news.google.com/read/CBMimAFBVV95cUxPb...
3,tesla,Tesla chair says the company could be on the c...,https://news.google.com/read/CBMixAFBVV95cUxNV...
4,tesla,Elon Musk's pay package is a clear signal that...,https://news.google.com/read/CBMiygFBVV95cUxNQ...
5,tesla,"Cathie Wood Sells Tesla (TSLA) Stock, and Buys...",https://news.google.com/read/CBMinwFBVV95cUxNd...
6,tesla,Tesla's China sales fall to 3-year low amid te...,https://news.google.com/read/CBMiugFBVV95cUxPb...
7,tesla,Tesla’s Cybertruck Head Exits. What It Means f...,https://news.google.com/read/CBMi0AFBVV95cUxQa...
8,tesla,Tesla is still losing ground in the world’s bi...,https://news.google.com/read/CBMiqwNBVV95cUxOX...
9,tesla,Head Of Tesla’s Controversial Cybertruck Unit ...,https://news.google.com/read/CBMivwFBVV95cUxPS...
